In [ ]:
!pip install transformers
!pip install datasets
!pip install trl
!pip install nltk
!pip install accelerate

In [42]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install "unsloth[colab_ampere] @ git+https://github.com/unslothai/unsloth.git"
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
pass

!pip install "git+https://github.com/huggingface/transformers.git" # Native 4bit loading works!

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TrainingArguments
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [2]:
from unsloth import FastLanguageModel
import nltk
from nltk.corpus import wordnet
import ssl

In [3]:
'''
def LoadModelHUB():
    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
    tokenizer.padding_side = 'right' # to avoid the future warning
    model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
    return model, tokenizer
'''
def formatting_prompts_func(sentences):
    out_sentences = []
    for sentence in sentences['sentences']:
        for text in sentence['sentence']:
            out_sentences.append(text)

    return out_sentences


In [4]:
def LoadModelUnsloth():
    max_seq_length = 2048
    model, tokenizer = FastLanguageModel.from_pretrained(
            "mistralai/Mistral-7B-v0.1", device_map="auto",
            max_seq_length = max_seq_length, dtype=None, load_in_4bit=True)


    model = FastLanguageModel.get_peft_model(
                    model,
                    r = 16,
                    target_modules = ["q_proj", "k_proj", "v_proj",
                        "o_proj", "gate_proj", "up_proj", "down_proj",],
                    lora_alpha = 16,
                    lora_dropout = 0, # Supports any, but = 0 is optimized
                    bias = "none",    # Supports any, but = "none" is optimized
                    use_gradient_checkpointing = True,
                    random_state = 3407,
                    use_rslora = False,  # We support rank stabilized LoRA
                    loftq_config = None, # And LoftQ
                    )

    return model, tokenizer

In [5]:
def LoadDataset():
    dataset = load_dataset("McGill-NLP/stereoset", "intersentence", split='validation')
    #dataset = dataset.map(formatting_prompts_func, remove_columns=[f for f in dataset.features if not f == 'sentences'],batched=True)
    return dataset

In [6]:
import torch
def FineTuneModel(model, tokenizer, dataset):
    trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    formatting_func=formatting_prompts_func,
    max_seq_length = 2048,
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

    trainer.train()

In [7]:
def InitNLTK():
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

    nltk.download('wordnet')
    nltk.download('omw-1.4')

In [8]:
def GetSynonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

In [28]:
def InitBiasWords():
    '''
    Initialize the bias words in a list from a file
    '''
    bias_words = ['black','bad', 'evil', 'inferior', 'weak', 'worthless', 'disgusting',
                'stupid', 'ignorant', 'lazy', 'aggressive', 'dangerous', 'violent', 'criminal',
                'savage', 'uncivilized', 'barbaric', 'dirty', 'ugly', 'fat', 'skinny',
                'anorexic', 'obese', 'old', 'poor', 'cheap', 'greedy', 'selfish',
                'ungrateful', 'primitive', 'backwards', 'uneducated', 'uncultured', 'insane',
                'mad', 'crazy', 'lunatic', 'delusional', 'paranoid', 'hysterical']


    return bias_words

In [36]:
def MitigateBias(text, bias_words):
    words = text.split()
    new_words = []
    for word in words:
        synonyms = GetSynonyms(word)
        if word in bias_words:
            unbiased_synonyms = [syn for syn in synonyms if syn not in bias_words]
            if unbiased_synonyms:
                new_word = unbiased_synonyms[0]
                new_words.append(new_word)
            else:
                new_words.append(word)
        else:
            new_words.append(word)

    return ' '.join(new_words)

In [37]:

def HandleQuestion(model, tokenizer, dataset, bias_words):

    pipe = pipeline("text-generation", model=model, tokenizer = tokenizer)
    while True:
        question = input("Your question: ")
        if question=='quit':
            print("Bye")
            break

        print("finding a good answer for your question, please wait,...\n")


        outputs = pipe(question, do_sample=True, max_new_tokens=100,
                temperature=0.7, top_k=50, top_p=0.95, num_return_sequences=1)
        answers=outputs[0]['generated_text']
        print("Before mitigation of bias: ")
        print(answers)
        print("After mitigation of bias: ")
        print(MitigateBias(answers,bias_words))


'''
    if not bias_words:
        output = answers[0]['generated_text']
    else:
        output = MitigateBias(answers[0]['generated_text'],bias_words)

    print(output)
'''

"\n    if not bias_words:\n        output = answers[0]['generated_text']\n    else:\n        output = MitigateBias(answers[0]['generated_text'],bias_words)\n\n    print(output)\n"

In [38]:
def main():
    InitNLTK() # to remove the bias words
    bias_words = InitBiasWords() # list of bias words present in our dataset
    model, tokenizer = LoadModelUnsloth() # Load the mistrel 7b ai model
    dataset = LoadDataset() # load our dataset
    FineTuneModel(model, tokenizer, dataset)

    HandleQuestion(model, tokenizer, dataset, bias_words)

In [ ]:
if __name__ == "__main__":
    main()